---
title: "Homework 2 - Scrapy"
author: "Andrew Han"
date: "2024-02-05"
categories: [week 5, homework]
---

# Introduction

https://andrewshan214.github.io/PIC16B/posts/Homework%20#2%20-%20Scrapy/index.ipynb

In this blog post, I will be giving a tutorial on how to write a spider in scrapy to scrape information from a movie database. More specifically, I will be demonstrating using Christopher Nolan's "The Dark Knight", one of my favorite movies. Using the data that I scrape, I will create a chart to suggest movies based on the actors in "The Dark Knight".

# Writing the movie database parse() function

First we write the initial parse() function, shown below:

In [ ]:
def parse(self, response):
        cast_crew_url = f"https://www.themoviedb.org/movie/155-the-dark-knight/cast"
        yield scrapy.Request(cast_crew_url, callback=self.parse_full_credits)

This method works by saving the url of the full cast and crew page of the movie website. I use scrapy's built in functions to go into this website and call a second parse function that will save all of the nanmes of those that acted in the movie. The implementation of the second parse function is below.

# parse_full_credits function implementation

In [ ]:
def parse_full_credits(self, response):
        for cast_member in response.css("ol.people.credits li"):
            actor_url = cast_member.css('a::attr(href)').get()

            if actor_url:
                yield scrapy.Request(url=response.urljoin(actor_url), callback=self.parse_actor_page)

Firstly, I iterated through all of the actors by using the css identifier of only actors by looking at the html code in the movie cast website. While iterating, I saved each actor's unique page url in a variable. Using that url variable, I used Scrapy's request function to go into the actor's page. 

In this page, I called a third parse function.

# parse_actor_page function implementation

In [ ]:
def parse_actor_page(self, response):
        actor_name = response.css('div.title h2.title a::text').get()

        acting_roles = response.css('div.credits_list h3.zero:contains("Acting")')
        
        if acting_roles:
            for acting_role in acting_roles.xpath('./following-sibling::table[@class="card credits"]//table[@class="credit_group"]'):
                movie_name = acting_role.css('td.role a.tooltip bdi::text').get()

                if movie_name:
                    yield {
                        'actor': actor_name,
                        'movie_or_TV_name': movie_name,
                    }

I implemented this by first getting the actor's name by using the css identifier from the website.

Next, I sorted through only this actor's acting roles by using the css identifier for acting roles, which was "Zero". I iterated through all of these acting roles, and got each movie name, and yielded them into a dictionary of the actor's name and the movies they've acted in.